<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://rstudio-pubs-static.s3.amazonaws.com/256588_57b585da6c054349825cba46685d8464.html

https://cran.r-project.org/web/packages/easyPubMed/vignettes/getting_started_with_easyPubMed.html

http://r-statistics.co/Association-Mining-With-R.html

https://blog.revolutionanalytics.com/2019/02/sequential-pattern-mining-in-r.html

In [ ]:
version

In [ ]:
install.packages("easyPubMed")
install.packages("tm")
install.packages("qdap")

In [ ]:
install.packages("arules")

In [ ]:
library(easyPubMed)
my_query <- '(("2020"[Date - Create]))'
my_entrez_id <- get_pubmed_ids(my_query)
my_abstracts_xml <- fetch_pubmed_data(my_entrez_id,retstart = 0,  format = "xml", retmax = 500)
my_PM_list <- articles_to_list(pubmed_data = my_abstracts_xml)

In [ ]:
xx <- lapply(my_PM_list, article_to_df, autofill = TRUE, max_chars = 50)
full_df <- do.call(rbind, xx)

In [ ]:
by_year_month<-full_df[seq(1, 500, by = 1), c("year", "month", "title")] 
by_year<-full_df[seq(1, 500, by = 1), c("year", "title")] 

In [ ]:
head(by_year)
head(by_year_month)

In [ ]:
library(tm)
text_vec <- VectorSource(by_year$title)
vec_corpus <- VCorpus(text_vec)


In [ ]:
str(vec_corpus)

In [ ]:
library(qdap)
frequent_terms <- freq_terms(by_year$title, 30)
plot(frequent_terms)

In [ ]:
clean_corpus <- function(corpus){
  corpus <- tm_map(corpus, stripWhitespace)
  corpus <- tm_map(corpus, removePunctuation)
  corpus <- tm_map(corpus, removeNumbers)
  corpus <- tm_map(corpus, content_transformer(tolower))
  corpus <- tm_map(corpus, removeWords, stopwords("en"))
    return(corpus)
}
# Apply your customized function to the tweet_corp: clean_corp
clean_corp <- clean_corpus(vec_corpus)
clean_corp

In [ ]:
clean_corp[[227]][1]
by_year$title[227]

In [ ]:
# Create the dtm from the corpus: 
pubmed_dtm <- DocumentTermMatrix(clean_corp)
# Print out tweets_dtm data
pubmed_dtm

In [ ]:


# Since the sparsity is so high, i.e. a proportion of cells with 0s/ cells with other values is too large,
# let's remove some of these low frequency terms
pubmed_dtm_rm_sparse <- removeSparseTerms(pubmed_dtm, 0.98)

# Convert tweets_dtm to a matrix: tweets_m
pubmed_m <- as.matrix(pubmed_dtm_rm_sparse)
# Print the dimensions of tweets_m
dim(pubmed_m)
# Review a portion of the matrix
pubmed_m[0:10, 0:100]

In [ ]:
library(arules)

rules <- apriori (pubmed_m, parameter = list(supp = 0.001, conf = 0.5))

In [ ]:

rules_conf <- sort (rules, by="confidence", decreasing=TRUE) # 'high-confidence' rules.
rules_conf
inspect(head(rules_conf))

In [ ]:
rules_lift <- sort (rules, by="lift", decreasing=TRUE) # 'high-lift' rules.
inspect(head(rules_lift))